In [7]:
import pandas as pd
import requests
import os


In [9]:
import sys
!{sys.executable} -m pip install openpyxl

You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
#get listed companies 
url = 'https://dps.psx.com.pk/market-watch'
res = requests.get(url)
dfs = pd.read_html(res.text)
df = dfs[0]
companies = list(df['SYMBOL'])

In [4]:
df_bargain = pd.DataFrame(columns = ['SYMBOL','HIGH','LOW','CURRENT'])
df_notbargain = pd.DataFrame(columns = ['SYMBOL','HIGH','LOW','CURRENT'])
no_data = []
i = 1
for company in companies:
    url_company = os.path.join('https://dps.psx.com.pk/timeseries/eod/',company)
    res_company = requests.get(url_company).json()
    df = pd.DataFrame(res_company['data'],columns = ['Date','Price','Unknown1','Unknown2'])
    #data prepeocessing
    df.drop(['Unknown1','Unknown2'],inplace = True,axis = 1)
    df['Date'] = pd.to_datetime(df['Date'],unit = 's')
    df['Price'] = pd.to_numeric(df['Price'])
    #anlyzing current stock price wrt highest and lowest price
    try:
        max_price = df.iloc[df['Price'].idxmax()]['Price']
        min_price = df.iloc[df['Price'].idxmin()]['Price']
        current_price = df.iloc[0]['Price']
        #percentage difference between current stock price and the lowest price wrt max price
        lowest_price_ratio = (min_price/max_price)*100
        current_price_ratio = (current_price/max_price)*100
        difference = abs(current_price_ratio-lowest_price_ratio)
        if difference < 10:
            df_bargain = df_bargain.append({'SYMBOL': company,'HIGH':max_price,'LOW':min_price,'CURRENT':current_price},ignore_index=True)
        else:
            df_notbargain = df_notbargain.append({'SYMBOL': company,'HIGH':max_price,'LOW':min_price,'CURRENT':current_price},ignore_index=True)
        print(i,' comapany analyzed')
        i+=1
    except:
        no_data.append(company)
        print('no data found for ', company)
    finally:
        continue

1  comapany analyzed
no data found for  HASCOLDEF
2  comapany analyzed
3  comapany analyzed
4  comapany analyzed
5  comapany analyzed
6  comapany analyzed
7  comapany analyzed
8  comapany analyzed
9  comapany analyzed
no data found for  NBPXD
10  comapany analyzed
11  comapany analyzed
12  comapany analyzed
13  comapany analyzed
14  comapany analyzed
15  comapany analyzed
16  comapany analyzed
17  comapany analyzed
18  comapany analyzed
19  comapany analyzed
20  comapany analyzed
no data found for  FABLXD
21  comapany analyzed
no data found for  DSLDEF
22  comapany analyzed
23  comapany analyzed
24  comapany analyzed
25  comapany analyzed
26  comapany analyzed
27  comapany analyzed
28  comapany analyzed
29  comapany analyzed
30  comapany analyzed
31  comapany analyzed
32  comapany analyzed
33  comapany analyzed
34  comapany analyzed
35  comapany analyzed
no data found for  ATRLXD
36  comapany analyzed
37  comapany analyzed
38  comapany analyzed
39  comapany analyzed
40  comapany analyz

In [4]:
df_bargain.head(15)

,SYMBOL,HIGH,LOW,CURRENT
0,BAFL,58.94,28.55,32.82
1,UNITY,188.66,7.62,21.16
2,CNERGY,18.12,4.71,5.04
3,OGDC,177.31,75.01,78.40
4,ENGRO,366.00,226.14,229.42
5,HBL,218.99,83.15,83.15
6,PPL,222.96,64.91,65.08
7,DGKC,171.58,42.31,59.30
8,GTECH,13.37,7.30,8.20
9,FCCL,35.21,11.68,15.00


In [5]:
df_bargain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SYMBOL   103 non-null    object 
 1   HIGH     103 non-null    float64
 2   LOW      103 non-null    float64
 3   CURRENT  103 non-null    float64
dtypes: float64(3), object(1)
memory usage: 3.3+ KB


In [10]:
file_name = 'potential_bargains.xlsx'
df_bargain.to_excel(file_name)